In [1]:
import pandas as pd
import numpy as np
import re
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
import matplotlib
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import nltk
import nltk.data
import seaborn as sns
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
nltk.download("punkt")
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")

# import nltk
nltk.download('stopwords')
# !pip install --user -U nltk
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

import warnings
warnings.filterwarnings('ignore')


C:\Users\AKSHITA\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AKSHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AKSHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AKSHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\AKSHITA\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AKSHITA\AppData\Roaming\

In [30]:
all_docs=pd.read_csv("C:/Users/AKSHITA/OneDrive/Desktop/IIITD/Sem6/IR/CSE508_Winter2023_A2_104/Q2/all_features.csv")

In [31]:
all_docs=all_docs.drop(['Unnamed: 0'],axis=1)
x=all_docs.iloc[:,0:85735]
y=all_docs.iloc[:,85735]                  

In [7]:
#splitting dataset into 70-30 train-test resp.

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=125)

Training the Naive Bayes classifier with TF-ICF

In [8]:
#implementing naive_bayes with tf_icf representation of the docs
model = GaussianNB()
model.fit(x_train, y_train)


GaussianNB()

In [41]:
# business = 0
# tech = 1
# politics = 2
# sport = 3
# entertainment = 4
#probability of each category based on the frequency of documents in the training set that belong to that category

category_freq=[]
for i in range(0,5):
    category_freq.append(len(y_train[y_train==i]))
# print(category_freq)

prob_dict={'business':round(category_freq[0]/len(y_train),3),'tech':round(category_freq[1]/len(y_train),3),'politics':round(category_freq[2]/len(y_train),3),'sport':round(category_freq[3]/len(y_train),3),'entertainment':round(category_freq[4]/len(y_train),3)}
print(prob_dict)

{'business': 0.219, 'tech': 0.173, 'politics': 0.185, 'sport': 0.239, 'entertainment': 0.185}


In [36]:
#probability of each feature given each category based on the TF-ICF values of that feature in documents belonging to that category.

def preprocessing(text):
    # text=text[0:100]
    #Lowercasing the text
    lower=text.lower()

    #Splitting into tokens
    tokens=lower.split()

    #Removing stop words
    without_stop = []
    for tok in tokens:
        if tok not in en_stopwords:
            without_stop.append(tok)

    #Removing Punctuations and Blank Space tokens
    tokenizer = RegexpTokenizer(r"\w+")
    not_final=tokenizer.tokenize(' '.join(without_stop))

    ps = PorterStemmer()
    after_stem = []
    for word in not_final:
        stem_word = ps.stem(word)
        after_stem.append(stem_word)
    
    wnl = WordNetLemmatizer()

    final = []
    for word in after_stem:
        fin_word = wnl.lemmatize(word, pos="v")
        final.append(fin_word)

    return final


tf_icf=pd.read_csv("C:/Users/AKSHITA/OneDrive/Desktop/IIITD/Sem6/IR/CSE508_Winter2023_A2_104/Q2/tf_icf_matrix.csv")
def prob_feat_category(term,category):
    dic={'business' :0, 'tech': 1, 'politics' : 2, 'sport' : 3, 'entertainment' : 4}
    return tf_icf[preprocessing(term)[0]][dic[category]]/tf_icf.iloc[dic[category]].sum()

print(prob_feat_category('bernie','business'))
print(prob_feat_category('fraud','tech'))

0.0008245823429566968
0.00018628266934289073


Testing the Naive Bayes classifier

In [51]:
print("accuracy with all features 70-30 split: ",round(sklearn.metrics.accuracy_score(model.predict(x_test), y_test),5))
print("precision with all features 70-30 split: ",round(sklearn.metrics.precision_score(model.predict(x_test), y_test,average='macro'),5))
print("recall with all features 70-30 split: ",round(sklearn.metrics.recall_score(model.predict(x_test), y_test,average='macro'),5))
print("f1_score with all features 70-30 split: ",round(sklearn.metrics.f1_score(model.predict(x_test), y_test,average='macro'),5))
model.predict(x_test)

accuracy with all features 70-30 split:  0.97763
precision with all features 70-30 split:  0.97748
recall with all features 70-30 split:  0.97704
f1_score with all features 70-30 split:  0.97704


array([3, 1, 0, 3, 1, 0, 2, 1, 1, 4, 2, 4, 4, 2, 3, 1, 2, 0, 4, 1, 4, 2,
       4, 4, 0, 3, 1, 4, 3, 0, 2, 0, 0, 3, 1, 2, 4, 2, 4, 4, 3, 2, 0, 1,
       1, 3, 2, 0, 1, 0, 4, 3, 4, 1, 4, 0, 0, 3, 0, 3, 2, 0, 0, 4, 3, 1,
       3, 4, 2, 2, 4, 0, 1, 3, 0, 3, 0, 2, 4, 0, 1, 0, 4, 4, 3, 1, 1, 0,
       1, 0, 0, 0, 2, 4, 1, 0, 0, 1, 1, 4, 0, 1, 2, 3, 2, 2, 0, 4, 2, 1,
       0, 4, 4, 3, 1, 2, 3, 3, 4, 1, 1, 3, 2, 0, 0, 3, 3, 2, 0, 1, 0, 1,
       1, 3, 2, 0, 0, 0, 1, 1, 3, 4, 3, 3, 3, 4, 4, 2, 4, 0, 0, 3, 1, 0,
       0, 3, 3, 4, 1, 4, 1, 0, 3, 1, 4, 3, 1, 4, 4, 2, 2, 1, 3, 2, 0, 0,
       3, 1, 3, 0, 0, 0, 4, 4, 0, 3, 0, 3, 3, 1, 2, 3, 2, 3, 3, 4, 4, 0,
       1, 0, 0, 4, 1, 4, 2, 2, 0, 2, 3, 0, 3, 4, 0, 2, 4, 1, 2, 2, 3, 1,
       0, 2, 1, 0, 4, 2, 0, 1, 3, 4, 0, 0, 0, 1, 3, 3, 2, 0, 2, 2, 0, 3,
       4, 1, 4, 2, 4, 3, 3, 4, 4, 4, 0, 0, 1, 3, 2, 4, 4, 1, 0, 3, 4, 0,
       3, 1, 1, 3, 0, 1, 4, 1, 4, 4, 4, 2, 0, 1, 3, 1, 3, 2, 0, 2, 0, 0,
       1, 0, 2, 3, 2, 4, 1, 1, 2, 3, 2, 1, 2, 0, 1,

Improving the classifier

In [4]:
performance_matrix=pd.DataFrame()

In [9]:
# tf_icf features, 50-50 split

x_train_50, x_test_50, y_train_50, y_test_50 = train_test_split(x, y, test_size=0.5, random_state=125)
model50 = GaussianNB()
model50.fit(x_train_50, y_train_50)
temp50=[]
temp50.extend([round(sklearn.metrics.accuracy_score(model50.predict(x_test_50), y_test_50),5),round(sklearn.metrics.precision_score(model50.predict(x_test_50), y_test_50,average='macro'),5),round(sklearn.metrics.recall_score(model50.predict(x_test_50), y_test_50,average='macro'),5),round(sklearn.metrics.f1_score(model50.predict(x_test_50), y_test_50,average='macro'),5)])
performance_matrix['tf_icf_50_50']=temp50
performance_matrix

,tf_icf_50_50
0,0.96644
1,0.96691
2,0.96369
3,0.96464


In [10]:
# tf_icf features, 60-40 split

x_train_60, x_test_60, y_train_60, y_test_60 = train_test_split(x, y, test_size=0.4, random_state=125)
model60 = GaussianNB()
model60.fit(x_train_60, y_train_60)
temp60=[]
temp60.extend([round(sklearn.metrics.accuracy_score(model60.predict(x_test_60), y_test_60),5),round(sklearn.metrics.precision_score(model60.predict(x_test_60), y_test_60,average='macro'),5),round(sklearn.metrics.recall_score(model60.predict(x_test_60), y_test_60,average='macro'),5),round(sklearn.metrics.f1_score(model60.predict(x_test_60), y_test_60,average='macro'),5)])
performance_matrix['tf_icf_60_40']=temp60
# performance_matrix

In [11]:
temp70=[]
temp70.extend([round(sklearn.metrics.accuracy_score(model.predict(x_test), y_test),5),round(sklearn.metrics.precision_score(model.predict(x_test), y_test,average='macro'),5),round(sklearn.metrics.recall_score(model.predict(x_test), y_test,average='macro'),5),round(sklearn.metrics.f1_score(model.predict(x_test), y_test,average='macro'),5)])
performance_matrix['tf_icf_70_30']=temp70
# performance_matrix

In [12]:
# tf_icf features, 80-20 split

x_train_80, x_test_80, y_train_80, y_test_80 = train_test_split(x, y, test_size=0.2, random_state=125)
model80 = GaussianNB()
model80.fit(x_train_80, y_train_80)
temp80=[]
temp80.extend([round(sklearn.metrics.accuracy_score(model80.predict(x_test_80), y_test_80),5),round(sklearn.metrics.precision_score(model80.predict(x_test_80), y_test_80,average='macro'),5),round(sklearn.metrics.recall_score(model80.predict(x_test_80), y_test_80,average='macro'),5),round(sklearn.metrics.f1_score(model80.predict(x_test_80), y_test_80,average='macro'),5)])
performance_matrix['tf_icf_80_20']=temp80

In [13]:
# tf_icf features, 80-20 split, 20 best features

k_best_20 = SelectKBest(k=20)
k_best_20.fit(x_train_80,y_train_80)
model_icf_20 = GaussianNB()
model_icf_20.fit(k_best_20.transform(x_train_80),y_train_80)
temp_icf_20=[]
temp_icf_20.extend([round(sklearn.metrics.accuracy_score(model_icf_20.predict(k_best_20.transform(x_test_80)), y_test_80),5),round(sklearn.metrics.precision_score(model_icf_20.predict(k_best_20.transform(x_test_80)), y_test_80,average='macro'),5),round(sklearn.metrics.recall_score(model_icf_20.predict(k_best_20.transform(x_test_80)), y_test_80,average='macro'),5),round(sklearn.metrics.f1_score(model_icf_20.predict(k_best_20.transform(x_test_80)), y_test_80,average='macro'),5)])
performance_matrix['tf_icf_20']=temp_icf_20
# performance_matrix

In [14]:
# tf_icf features, 80-20 split, 200 best features

k_best_200 = SelectKBest(k=200)
k_best_200.fit(x_train_80,y_train_80)
model_icf_200 = GaussianNB()
model_icf_200.fit(k_best_200.transform(x_train_80),y_train_80)
temp_icf_200=[]
temp_icf_200.extend([round(sklearn.metrics.accuracy_score(model_icf_200.predict(k_best_200.transform(x_test_80)), y_test_80),5),round(sklearn.metrics.precision_score(model_icf_200.predict(k_best_200.transform(x_test_80)), y_test_80,average='macro'),5),round(sklearn.metrics.recall_score(model_icf_200.predict(k_best_200.transform(x_test_80)), y_test_80,average='macro'),5),round(sklearn.metrics.f1_score(model_icf_200.predict(k_best_200.transform(x_test_80)), y_test_80,average='macro'),5)])
performance_matrix['tf_icf_200']=temp_icf_200
performance_matrix

,tf_icf_50_50,tf_icf_60_40,tf_icf_70_30,tf_icf_80_20,tf_icf_20,tf_icf_200
0,0.96644,0.96812,0.97763,0.97987,0.51678,0.86242
1,0.96691,0.96834,0.97748,0.97934,0.54471,0.86584
2,0.96369,0.96727,0.97704,0.97962,0.57862,0.87408
3,0.96464,0.96759,0.97704,0.97930,0.50034,0.86444


In [34]:
tf_idf=pd.read_csv("C:/Users/AKSHITA/OneDrive/Desktop/IIITD/Sem6/IR/CSE508_Winter2023_A2_104/Q2/q2_tf_idf.csv")

In [51]:
for_labels=pd.read_csv("C:/Users/AKSHITA/OneDrive/Desktop/IIITD/Sem6/IR/CSE508_Winter2023_A2_104/Q2/BBC News Train.csv")

In [52]:
y2=for_labels['Category']
for i in range (len(y)):
    dic={'business' :0, 'tech': 1, 'politics' : 2, 'sport' : 3, 'entertainment' : 4}
    y2[i]=dic[y2[i]]
y2=y2.astype(int)

In [16]:
tf_idf

,worldcom,ex,boss,launch,defenc,lawyer,defend,former,chief,berni,...,610m,wharf,glocer,6gb,4gb,santi,unwelcom,defac,phpbb,randomli
0,0.151494,0.032443,0.012444,0.009402,0.014795,0.045548,0.012523,0.016119,0.024481,0.026068,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.011505,0.000000,0.000000,0.028074,0.011578,0.029805,0.015089,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011877,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1486,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1487,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010386,0.000000,...,0.042944,0.042944,0.042944,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1488,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.020041,0.020041,0.00000,0.000000,0.000000,0.000000,0.000000


In [65]:
# tf_idf features, 80-20 split

x_train_idf, x_test_idf, y_train_idf, y_test_idf = train_test_split(tf_idf, y2, test_size=0.20, random_state=125)
modelidf = GaussianNB()
modelidf.fit(x_train_idf, y_train_idf)
tempidf=[]
y_pred_idf=modelidf.predict(x_test_idf)
tempidf.extend([round(sklearn.metrics.accuracy_score(y_pred_idf, y_test_idf),5),round(sklearn.metrics.precision_score(y_pred_idf, y_test_idf,average='macro'),5),round(sklearn.metrics.recall_score(y_pred_idf, y_test_idf,average='macro'),5),round(sklearn.metrics.f1_score(y_pred_idf, y_test_idf,average='macro'),5)])
performance_matrix['tf_idf_80_20_all']=tempidf
performance_matrix

,tf_icf_50_50,tf_icf_60_40,tf_icf_70_30,tf_icf_80_20,tf_icf_20,tf_icf_200,tf_idf_80_20_all
0,0.96644,0.96812,0.97763,0.97987,0.51678,0.86242,0.88255
1,0.96691,0.96834,0.97748,0.97934,0.54471,0.86584,0.88213
2,0.96369,0.96727,0.97704,0.97962,0.57862,0.87408,0.88493
3,0.96464,0.96759,0.97704,0.97930,0.50034,0.86444,0.88278


In [66]:
# tf_idf features, 70-30 split

x_train_idf, x_test_idf, y_train_idf, y_test_idf = train_test_split(tf_idf, y2, test_size=0.30, random_state=125)
modelidf = GaussianNB()
modelidf.fit(x_train_idf, y_train_idf)
tempidf=[]
y_pred_idf=modelidf.predict(x_test_idf)
tempidf.extend([round(sklearn.metrics.accuracy_score(y_pred_idf, y_test_idf),5),round(sklearn.metrics.precision_score(y_pred_idf, y_test_idf,average='macro'),5),round(sklearn.metrics.recall_score(y_pred_idf, y_test_idf,average='macro'),5),round(sklearn.metrics.f1_score(y_pred_idf, y_test_idf,average='macro'),5)])
performance_matrix['tf_idf_70_30_all']=tempidf
performance_matrix

,tf_icf_50_50,tf_icf_60_40,tf_icf_70_30,tf_icf_80_20,tf_icf_20,tf_icf_200,tf_idf_80_20_all,tf_idf_70_30_all
0,0.96644,0.96812,0.97763,0.97987,0.51678,0.86242,0.88255,0.90157
1,0.96691,0.96834,0.97748,0.97934,0.54471,0.86584,0.88213,0.90196
2,0.96369,0.96727,0.97704,0.97962,0.57862,0.87408,0.88493,0.90231
3,0.96464,0.96759,0.97704,0.97930,0.50034,0.86444,0.88278,0.90200


In [72]:
#tf_idf, 70-30 split, 100 best features

k_best_200 = SelectKBest(k=100)
k_best_200.fit(x_train_idf,y_train_idf)
model_icf_200 = GaussianNB()
model_icf_200.fit(k_best_200.transform(x_train_idf),y_train_idf)
temp_icf_200=[]
temp_icf_200.extend([round(sklearn.metrics.accuracy_score(model_icf_200.predict(k_best_200.transform(x_test_idf)), y_test_idf),5),round(sklearn.metrics.precision_score(model_icf_200.predict(k_best_200.transform(x_test_idf)), y_test_idf,average='macro'),5),round(sklearn.metrics.recall_score(model_icf_200.predict(k_best_200.transform(x_test_idf)), y_test_idf,average='macro'),5),round(sklearn.metrics.f1_score(model_icf_200.predict(k_best_200.transform(x_test_idf)), y_test_idf,average='macro'),5)])
performance_matrix['tf_idf_100']=temp_icf_200
performance_matrix

,tf_icf_50_50,tf_icf_60_40,tf_icf_70_30,tf_icf_80_20,tf_icf_20,tf_icf_200,tf_idf_80_20_all,tf_idf_70_30_all,tf_idf_200,tf_idf_400,tf_idf_300,tf_idf_100
0,0.96644,0.96812,0.97763,0.97987,0.51678,0.86242,0.88255,0.90157,0.91275,0.88814,0.90157,0.85459
1,0.96691,0.96834,0.97748,0.97934,0.54471,0.86584,0.88213,0.90196,0.91096,0.88765,0.90064,0.85589
2,0.96369,0.96727,0.97704,0.97962,0.57862,0.87408,0.88493,0.90231,0.91314,0.88785,0.89948,0.85876
3,0.96464,0.96759,0.97704,0.97930,0.50034,0.86444,0.88278,0.90200,0.91169,0.88711,0.89986,0.85611


In [73]:
performance_matrix.index=['Accuracy','Precision','Recall','F1-Score']
performance_matrix

,tf_icf_50_50,tf_icf_60_40,tf_icf_70_30,tf_icf_80_20,tf_icf_20,tf_icf_200,tf_idf_80_20_all,tf_idf_70_30_all,tf_idf_200,tf_idf_400,tf_idf_300,tf_idf_100
Accuracy,0.96644,0.96812,0.97763,0.97987,0.51678,0.86242,0.88255,0.90157,0.91275,0.88814,0.90157,0.85459
Precision,0.96691,0.96834,0.97748,0.97934,0.54471,0.86584,0.88213,0.90196,0.91096,0.88765,0.90064,0.85589
Recall,0.96369,0.96727,0.97704,0.97962,0.57862,0.87408,0.88493,0.90231,0.91314,0.88785,0.89948,0.85876
F1-Score,0.96464,0.96759,0.97704,0.97930,0.50034,0.86444,0.88278,0.90200,0.91169,0.88711,0.89986,0.85611
